In [3]:
def target():
    print("Running target()") 

In [10]:
def deco(func):
    def inner():
        print('running inner()')
    return inner # returns this


In [5]:
target()

Running target()


In [7]:
deco(target)()

running inner()


In [8]:
## the upper function have a same syntex as 

@deco
def target():
    print('Running target()')



In [9]:
target()

running inner()


In [13]:
registry = [] 

def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print("running f2()")

def f3():
    print("running f3()")


def main():
    print("running main()")
    ##decorator work as soon as imported
    print('registry ->', registry)
    f1()
    f2()
    f3()
    # print('registry ->', registry)

running register(<function f1 at 0x114afdda0>)
running register(<function f2 at 0x114afdd00>)


In [14]:
main()

running main()
registry -> [<function f1 at 0x114afdda0>, <function f2 at 0x114afdd00>]
running f1()
running f2()
running f3()
registry -> [<function f1 at 0x114afdda0>, <function f2 at 0x114afdd00>]


In [15]:
class Averager():
    def __init__(self):
        self.series = [] 
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series )
        return total / len(self.series)

In [17]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [18]:
## how about functional?

def make_averager():
    series = []  ## binding

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [21]:
avg.__code__.co_varnames

('new_value', 'total')

In [22]:
avg.__code__.co_freevars

('series',)

In [25]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [29]:
def make_averager():
    count = 0 
    total = 0 

    def averager(new_value):
        # print(count) 
        nonlocal count, total 
        count += 1 ## ?? 
        total += new_value 
        return total / count 
    return averager 


avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [30]:
import time 

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0 
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked 


@clock
def make_n_length_list(n):
    return list(range(n))

In [35]:
make_n_length_list(10)

[0.00000337s] make_n_length_list(10) -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [38]:
@clock
def snooze(seconds):
    time.sleep(seconds)

snooze(.1)

[0.10506479s] snooze(0.1) -> None


In [39]:
@clock 
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [40]:
factorial(10)

[0.00000046s] factorial(1) -> 1
[0.00006121s] factorial(2) -> 2
[0.00007283s] factorial(3) -> 6
[0.00007887s] factorial(4) -> 24
[0.00008467s] factorial(5) -> 120
[0.00009054s] factorial(6) -> 720
[0.00009608s] factorial(7) -> 5040
[0.00010175s] factorial(8) -> 40320
[0.00010742s] factorial(9) -> 362880
[0.00011442s] factorial(10) -> 3628800


3628800

In [41]:
## imporved clock 
import functools 

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter() 
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0 
        name = func.__name__ 
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result 
    return clocked
        

In [49]:
@clock 
def factorial(n, times = 1):
    print(times)
    return 1 if n < 2 else n * factorial(n-1, times=times)

In [50]:
factorial(10, times= 3)

3
3
3
3
3
3
3
3
3
3
[0.00000846s] factorial(1, times=3) -> 1
[0.00004813s] factorial(2, times=3) -> 2
[0.00007171s] factorial(3, times=3) -> 6
[0.00009100s] factorial(4, times=3) -> 24
[0.00010942s] factorial(5, times=3) -> 120
[0.00012921s] factorial(6, times=3) -> 720
[0.00014862s] factorial(7, times=3) -> 5040
[0.00016825s] factorial(8, times=3) -> 40320
[0.00018858s] factorial(9, times=3) -> 362880
[0.00027367s] factorial(10, times=3) -> 3628800


3628800

In [51]:
@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [52]:
fibonacci(10)

[0.00000029s] fibonacci(0) -> 0
[0.00000038s] fibonacci(1) -> 1
[0.00006083s] fibonacci(2) -> 1
[0.00000025s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00001238s] fibonacci(2) -> 1
[0.00002929s] fibonacci(3) -> 2
[0.00010262s] fibonacci(4) -> 3
[0.00000021s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00001108s] fibonacci(2) -> 1
[0.00002408s] fibonacci(3) -> 2
[0.00000021s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00001108s] fibonacci(2) -> 1
[0.00000021s] fibonacci(1) -> 1
[0.00000029s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00001125s] fibonacci(2) -> 1
[0.00002200s] fibonacci(3) -> 2
[0.00004383s] fibonacci(4) -> 3
[0.00007954s] fibonacci(5) -> 5
[0.00019329s] fibonacci(6) -> 8
[0.00000021s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000021s] fibonacci(1) -> 1
[0.00001075s] fibonacci(2) -> 1
[0.00002113s] fibonacci(3) -> 2
[0.00000021s] fibonacci(0) -> 0
[0.00000

55

In [58]:
@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [59]:
fibonacci(6)

[0.00000071s] fibonacci(0) -> 0
[0.00000067s] fibonacci(1) -> 1
[0.00018117s] fibonacci(2) -> 1
[0.00000100s] fibonacci(3) -> 2
[0.00020417s] fibonacci(4) -> 3
[0.00000087s] fibonacci(5) -> 5
[0.00022779s] fibonacci(6) -> 8


8

In [61]:
## use cahced function
fibonacci(7)

13

In [62]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

In [63]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [64]:
registry = [] 

def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')


print("running main()")
##decorator work as soon as imported
print('registry ->', registry)

running register(<function f1 at 0x114afe840>)
running main()
registry -> [<function f1 at 0x114afe840>]


In [69]:
registry = set()

def register(active=True):
    def decorate(func):
        print("Running register"
              f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate


@register(active=False)
def f1():
    print('running f1()')

@register(active=True)
def f2():
    print('running f2()')




print('registry ->', registry)
        

Running register(active=False)->decorate(<function f1 at 0x114c91760>)
Running register(active=True)->decorate(<function f2 at 0x114c91120>)
registry -> {<function f2 at 0x114c91120>}


In [77]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func): 
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0 
            name = func.__name__ 
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate



@clock
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

In [80]:
for i in range(3):
    snooze(.123)

In [76]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def target():\n    print("Running target()") ',
  '@decorate\ndef target():\n    print("Running target()") ',
  'def target():\n    print("Running target()") ',
  "def deco(func):\n    def inner():\n        print('running inner()')\n    return inner",
  'target()',
  'deco(target)',
  'deco(target)()',
  "## the upper function have a same syntex as \n\n@deco\ndef target():\n    print('Running target()')",
  'target()',
  "def deco(func):\n    def inner():\n        print('running inner()')\n    return inner # returns this",
  'registry = [] \n\ndef register(func):\n    print(f"running register({func})")\n    registry.append(func)\n    return func\n\n@register\ndef f1():\n    print(\'running f1()\')\n\n@register\

In [81]:
class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter() 
            _result = func(*_args) 
            elapsed = time.perf_counter() - t0 
            name = func.__name__ 
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked 
    

In [82]:
clk = clock() 

In [83]:
a = clk(snooze)

In [84]:
a(1)

[0.00000121s] decorate(1) -> <function clock.<locals>.decorate.<locals>.clocked at 0x1d4594220>


<function __main__.clock.<locals>.decorate.<locals>.clocked(*_args)>